In [1]:
import random
import queue

In [32]:
class AdjecencyList:

    class Vertex:
        def __init__(self):
            self.edges = set()

        def add_edge(self, edge):
            self.edges.add(edge)

    class Edge:
        def __init__(self, v1, v2):
            self.vertices = set((v1, v2))
            v1.add_edge(self)
            v2.add_edge(self)

    def __init__(self):
        self.V = set()
        self.E = set()

    def new_vertex(self):
        v = AdjecencyList.Vertex()
        self.V.add(v)
        return v

    def set_edge(self, v1, v2):
        assert v1 is not v2, 'Cycles not supported'
        e = AdjecencyList.Edge(v1, v2)
        self.E.add(e)

    def shortest_path(self, u, v):
        return self.bfs(u, end_at=v)[v]

    def bfs(self, u, end_at=None):
        visited = {u: 0}
        if u is not end_at:
            q = queue.Queue()
            q.put(u)
            while not q.empty():
                v = q.get()
                for edge in v.edges:
                    w = [w for w in edge.vertices if w is not v and w not in visited]
                    if len(w) == 0:
                        continue
                    assert len(w) == 1
                    visited[w[0]] = visited[v] + 1
                    q.put(w[0])
                    if end_at is not None and end_at == w[0]:
                        break
        return visited

    def contract(self):
        while len(self.V) > 2:
            e = random.choice(tuple(self.E))
            v1, v2 = e.vertices
            for v2edge in v2.edges:
                if v1 in v2edge.vertices: # remove loops
                    v1.edges.remove(v2edge)
                    self.E.remove(v2edge)
                else: # replace references to v2 by references to v1, and add edges to v1.edges
                    v2edge.vertices.remove(v2)
                    assert len(v2edge.vertices) == 1
                    v2edge.vertices.add(v1)
                    assert len(v2edge.vertices) == 2
                    v1.edges.add(v2edge)
            self.V.remove(v2) # remove vertex from vertex list
        a, _ = self.V
        return len(a.edges)

In [33]:
lines = []
with open('kargerMinCut.txt', 'r') as fp:
    for line in fp:
        l = tuple(int(vi)-1 for vi in line.split())
        lines.append(tuple(i for i in l if i >= l[0]))

def get_adjecency_list():
    adj_list = AdjecencyList()
    v = [adj_list.new_vertex() for _ in range(200)]
    for line in lines:
        for vi in line[1:]:
            adj_list.set_edge(v[line[0]], v[vi])
    return adj_list

In [4]:
# CONTRACTION
adj_list = get_adjecency_list()
m = adj_list.contract()
print("new min: %s" % m)
for _ in range(200): # should be N^2*LOG(n)
    adj_list = get_adjecency_list()
    mnew = adj_list.contract()
    if mnew < m:
        print("new min: %s < %s" % (mnew, m))
        m = mnew

new min: 20
new min: 17 < 20


In [5]:
# BFS
adj_list = get_adjecency_list()
v = random.choice(tuple(adj_list.V))
adj_list.bfs(v)

{<__main__.AdjecencyList.Vertex at 0x1120c0048>: 3,
 <__main__.AdjecencyList.Vertex at 0x1120a2080>: 2,
 <__main__.AdjecencyList.Vertex at 0x1120c0ac8>: 4,
 <__main__.AdjecencyList.Vertex at 0x1120c00b8>: 1,
 <__main__.AdjecencyList.Vertex at 0x1120c00f0>: 2,
 <__main__.AdjecencyList.Vertex at 0x1120a2588>: 1,
 <__main__.AdjecencyList.Vertex at 0x1120c0128>: 3,
 <__main__.AdjecencyList.Vertex at 0x1120c0160>: 2,
 <__main__.AdjecencyList.Vertex at 0x1120a2198>: 2,
 <__main__.AdjecencyList.Vertex at 0x1120a21d0>: 1,
 <__main__.AdjecencyList.Vertex at 0x1120a2208>: 3,
 <__main__.AdjecencyList.Vertex at 0x1120c0b00>: 2,
 <__main__.AdjecencyList.Vertex at 0x1120c0240>: 2,
 <__main__.AdjecencyList.Vertex at 0x1120a2278>: 3,
 <__main__.AdjecencyList.Vertex at 0x1120c05c0>: 3,
 <__main__.AdjecencyList.Vertex at 0x1120c02b0>: 1,
 <__main__.AdjecencyList.Vertex at 0x1120a22e8>: 2,
 <__main__.AdjecencyList.Vertex at 0x1120a9080>: 3,
 <__main__.AdjecencyList.Vertex at 0x1120a2320>: 3,
 <__main__.A

In [34]:
# SHORTEST PATH
adj_list = get_adjecency_list()
vertices = tuple(adj_list.V)
v = random.choice(vertices)
u = random.choice(vertices)
adj_list.shortest_path(u, v)

3

In [62]:
class DirectedGraph:

    class Vertex:

        def __init__(self, G):
            self.heads = set()
            self.tails = set()
            self.G = G

        def connect_to(self, v_name):
            v = self.G.vertex(v_name)
            if v not in self.heads and self not in v.tails:
                assert v is not self, 'cannot connect a vertex to itself directly'
                self.heads.add(v)
                v.tails.add(self)
            return self

    def __init__(self):
        self.V = dict()

    def vertex(self, v_name):
        if v_name not in self.V:
            self.V[v_name] = DirectedGraph.Vertex(self)
        return self.V[v_name]

    def dfs_topsort(self):
        seen = dict()
        label = len(self.V)
        def recurse(v):
            nonlocal label
            for h in v.heads:
                if h not in seen:
                    seen[h] = -1
                    recurse(h)
            seen[v] = label
            label -= 1
        for k in self.V:
            if self.V[k] not in seen:
                recurse(self.V[k])
        return sorted(((seen[self.V[k]], k) for k in self.V), key=lambda n: n[0])

In [63]:
g = DirectedGraph()
with open('kargerMinCut.txt', 'r') as fp:
    for line in fp:
        l = tuple(int(vi) for vi in line.split())
        t = g.vertex(l[0])
        for h in l[1:]:
            t.connect_to(h)

In [64]:
g.dfs_topsort()

[(0, 1),
 (1, 39),
 (2, 7),
 (3, 107),
 (4, 172),
 (5, 100),
 (6, 45),
 (7, 89),
 (8, 19),
 (9, 80),
 (10, 119),
 (11, 174),
 (12, 99),
 (13, 52),
 (14, 96),
 (15, 126),
 (16, 75),
 (17, 57),
 (18, 102),
 (19, 85),
 (20, 15),
 (21, 145),
 (22, 79),
 (23, 108),
 (24, 37),
 (25, 146),
 (26, 131),
 (27, 112),
 (28, 135),
 (29, 120),
 (30, 171),
 (31, 78),
 (32, 175),
 (33, 151),
 (34, 32),
 (36, 18),
 (37, 34),
 (38, 46),
 (39, 14),
 (40, 91),
 (41, 144),
 (42, 26),
 (43, 166),
 (44, 6),
 (45, 124),
 (46, 71),
 (47, 125),
 (48, 178),
 (49, 25),
 (50, 68),
 (51, 118),
 (52, 5),
 (53, 73),
 (54, 195),
 (55, 140),
 (56, 76),
 (57, 4),
 (58, 158),
 (59, 156),
 (60, 113),
 (61, 42),
 (62, 27),
 (63, 169),
 (64, 50),
 (65, 65),
 (66, 12),
 (67, 70),
 (68, 74),
 (69, 95),
 (70, 177),
 (71, 148),
 (72, 43),
 (73, 47),
 (74, 13),
 (75, 90),
 (76, 163),
 (77, 101),
 (78, 180),
 (79, 21),
 (80, 197),
 (81, 147),
 (82, 83),
 (83, 88),
 (84, 173),
 (85, 3),
 (86, 157),
 (87, 134),
 (88, 2),
 (89, 179)